In [3]:
import os

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\Image-Caption-Generator'

In [5]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
import tensorflow as tf

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

C:\Users\ramak\AppData\Local\Temp\ipykernel_15736\3300073956.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [6]:
def extract_features(directory):
        model = Xception( include_top=False, pooling='avg' )
        features = {}
        for img in tqdm(os.listdir(directory)):
            filename = directory + "/" + img
            image = Image.open(filename)
            image = image.resize((299,299))
            image = np.expand_dims(image, axis=0)
            #image = preprocess_input(image)
            image = image/127.5
            image = image - 1.0

            feature = model.predict(image)
            features[img] = feature
        return features

#2048 feature vector
path = os.path.join("artifact/data_ingestion/data/Images/")
features = extract_features(path)
dump(features, open("features.p","wb"))

C:\Users\ramak\AppData\Local\Temp\ipykernel_15736\3490669101.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for img in tqdm(os.listdir(directory)):


  0%|          | 0/8091 [00:00<?, ?it/s]

1/1 [==============================] - 1s 592ms/step


In [7]:
features

{'1000268201_693b08cb0e.jpg': array([[0.4734095 , 0.01730908, 0.07334223, ..., 0.08557981, 0.02102295,
         0.23765537]], dtype=float32),
 '1001773457_577c3a7d70.jpg': array([[0.00158197, 0.11114135, 0.0003743 , ..., 0.2650444 , 0.35281467,
         0.05871551]], dtype=float32),
 '1002674143_1b742ab4b8.jpg': array([[0.        , 0.0249318 , 0.01557386, ..., 0.        , 0.        ,
         0.10194069]], dtype=float32),
 '1003163366_44323f5815.jpg': array([[0.1456801 , 0.00272415, 0.27775803, ..., 0.17016964, 0.11957451,
         0.09415703]], dtype=float32),
 '1007129816_e794419615.jpg': array([[0.        , 0.12444558, 0.7391587 , ..., 0.00390426, 0.00997241,
         0.50171393]], dtype=float32),
 '1007320043_627395c3d8.jpg': array([[0.04138516, 0.        , 0.01274429, ..., 0.00944679, 0.64201516,
         0.04793233]], dtype=float32),
 '1009434119_febe49276a.jpg': array([[0.        , 0.        , 0.02635755, ..., 0.30525967, 0.23072195,
         0.14180997]], dtype=float32),
 '1012